In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)  
print(tf.executing_eagerly())

## Loading Dataset

In [ ]:
from tensorflow.keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(f'Training data shapes: X={X_train.shape}, Y={y_train.shape}')
print(f'Testing data shapes: X={X_test.shape}, Y={y_test.shape}')

for j in range(5):
    i = np.random.randint(0, X_train.shape[0])
    plt.subplot(550 + 1 + j)
    plt.imshow(X_train[i], cmap='gray')
    plt.title(y_train[i])
plt.show()

## Data Preparation

In [ ]:
train_data = X_train.astype('float32') / 255
test_data = X_test.astype('float32') /255

train_data = train_data.reshape(train_data.shape[0], 28, 28, 1)
test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

print(train_data.shape, test_data.shape)

## VAE

In [ ]:
input_data = tf.keras.layers.Input(shape=(28, 28, 1))

encoder = tf.keras.layers.Conv2D(64, (5,5), activation='relu')(input_data)
encoder = tf.keras.layers.MaxPooling2D((2,2))(encoder)

encoder = tf.keras.layers.Conv2D(64, (3,3), activation="relu")(encoder)
encoder = tf.keras.layers.MaxPooling2D((2,2))(encoder)

encoder = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(encoder)
encoder = tf.keras.layers.MaxPooling2D((2,2))(encoder)

encoder = tf.keras.layers.Flatten()(encoder)
encoder = tf.keras.layers.Dense(16)(encoder)

In [ ]:
from tensorflow.keras import layers
z_mean = layers.Dense(2, name='mean')(encoder)
z_log_var = layers.Dense(2, name='log_var')(encoder)

# Custom layer to add KL loss inside graph
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        epsilon = tf.random.normal(shape=tf.shape(z_mean))
        z = z_mean + tf.exp(0.5 * z_log_var) * epsilon
        # Add KL divergence to model.losses
        kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        self.add_loss(kl_loss)
        return z

latent_encoding = Sampling()([z_mean, z_log_var])

In [ ]:
encoder_model = tf.keras.Model(input_data, latent_encoding, name='Encoder')
encoder_model.summary()

In [ ]:
decoder_input = tf.keras.layers.Input(shape=(2,))
decoder = tf.keras.layers.Dense(64)(decoder_input)
decoder = tf.keras.layers.Reshape((1, 1, 64))(decoder)
decoder = tf.keras.layers.Conv2DTranspose(64, (3,3), activation='relu')(decoder)

decoder = tf.keras.layers.Conv2DTranspose(64, (3,3), activation='relu')(decoder)
decoder = tf.keras.layers.UpSampling2D((2, 2))(decoder)

decoder = tf.keras.layers.Conv2DTranspose(64, (3,3), activation='relu')(decoder)
decoder = tf.keras.layers.UpSampling2D((2,2))(decoder)

decoder_output = tf.keras.layers.Conv2DTranspose(1, (5,5), activation='sigmoid')(decoder)

In [ ]:
decoder_model = tf.keras.Model(decoder_input, decoder_output, name='Decoder')
decoder_model.summary()

## Combining Encoder and Decoder into VAE

In [ ]:
encoded = encoder_model(input_data)
decoded = decoder_model(encoded)

In [ ]:
autoencoder = tf.keras.models.Model(input_data, decoded, name='VAE')
autoencoder.summary()

## Loss Function (Reconstruction + KL Divergence)

In [ ]:
reconstruction_loss = tf.keras.losses.MeanSquaredError()

## Training the VAE

In [ ]:
autoencoder.compile(optimizer='adam', loss=reconstruction_loss)
autoencoder.summary()

In [ ]:
type(train_data)

In [ ]:
autoencoder.fit(train_data, train_data, epochs=1, batch_size=64, validation_data=(test_data, test_data))

In [ ]:
offset = 400
print('Real Test Images')

for i in range(9):
    plt.subplot(330 + 1 + i)
    plt.imshow(test_data[i+offset, :, :, 0], cmap='gray')
plt.show()

print('Reconstructed Test Images')
for i in range(9):
    plt.subplot(330 + 1 + i)
    output = autoencoder.predict(np.array([test_data[i+offset]]))
    op_image = np.reshape(output[0]*255, (28, 28))
    plt.imshow(op_image, cmap='gray')
plt.show()